In [1]:
import rasterio
import numpy as np
from scipy import ndimage

def fill_nodata_dem(dem_path, output_path):
    # Open DEM
    with rasterio.open(dem_path) as src:
        dem = src.read(1).astype(float)
        nodata = src.nodata
        profile = src.profile

    print(f"DEM NoData value: {nodata}")

    # Mask missing values
    if nodata is not None:
        mask = (dem == nodata)
    else:
        mask = np.isnan(dem)

    # Replace nodata with NaN for safe interpolation
    dem[mask] = np.nan

    # --- Interpolation using nearest valid neighbor (bilinear-like smoothing) ---
    # Create indices of nearest valid pixels
    filled = dem.copy()
    nan_mask = np.isnan(filled)

    # If there are still NaNs, fill them
    if np.any(nan_mask):
        # Find nearest neighbor indices
        nearest_x, nearest_y = ndimage.distance_transform_edt(
            nan_mask,
            return_indices=True
        )
        filled[nan_mask] = filled[nearest_x[nan_mask], nearest_y[nan_mask]]

    # Update metadata
    profile.update(dtype=rasterio.float32, nodata=None)

    # Save filled DEM
    with rasterio.open(output_path, "w", **profile) as dst:
        dst.write(filled.astype(np.float32), 1)

    print(f"✅ Missing values filled and saved at: {output_path}")


# Example usage
dem_file = r"C:\Users\Ankit\OneDrive\Desktop\merged_DEM_30m_32644_aligned.tif"
output_file = dem_file.replace(".tif", "_filled.tif")
fill_nodata_dem(dem_file, output_file)


DEM NoData value: 255.0


IndexError: boolean index did not match indexed array along dimension 0; dimension is 2 but corresponding boolean dimension is 11904